# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
import os
import time
# set working dir to same as env file
os.chdir('../../01_materials/labs/')


%load_ext dotenv
%dotenv


In [5]:
pip install dask

Note: you may need to restart the kernel to use updated packages.


In [6]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [8]:
import os
from glob import glob

# Write your code below.
ft_dir = os.getenv("PRICE_DATA")
parq_file = glob(os.path.join(ft_dir, "**/*.parquet"), recursive=True)
#df = dd.read_parquet(parq_file).compute().reset_index()

df = dd.read_parquet(parq_file).set_index("ticker")

In [9]:
df.compute()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year
ticker,,,,,,,,,
A,2024-01-02 00:00:00-05:00,137.710287,140.101950,137.431261,138.268341,1441600,0.0,0.0,2024
A,2024-01-03 00:00:00-05:00,137.520949,137.520949,130.615013,130.704697,2074500,0.0,0.0,2024
A,2024-01-04 00:00:00-05:00,130.096807,131.043507,129.738057,130.545242,2446600,0.0,0.0,2024
A,2024-01-05 00:00:00-05:00,129.548727,131.501930,128.173513,130.106781,1394000,0.0,0.0,2024
A,2024-01-08 00:00:00-05:00,129.688234,133.106335,129.359378,132.916992,1311400,0.0,0.0,2024
...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,2024
YUM,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,2024
YUM,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,2024


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [10]:
# Write your code below.

import numpy as np
dd_feat = (df.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1))
).assign(
    returns = lambda x: x['Close']/x['Close_lag'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))


C:\Users\klawa\AppData\Local\Temp\ipykernel_29736\1595277154.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (df.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [15]:
dd_feat.compute()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,
A,2024-01-02 00:00:00-05:00,137.710287,140.101950,137.431261,138.268341,1441600,0.0,0.0,2024,NaN,NaN,2.670689
A,2024-01-03 00:00:00-05:00,137.520949,137.520949,130.615013,130.704697,2074500,0.0,0.0,2024,138.268341,-0.054703,6.905936
A,2024-01-04 00:00:00-05:00,130.096807,131.043507,129.738057,130.545242,2446600,0.0,0.0,2024,130.704697,-0.001220,1.305450
A,2024-01-05 00:00:00-05:00,129.548727,131.501930,128.173513,130.106781,1394000,0.0,0.0,2024,130.545242,-0.003359,3.328417
A,2024-01-08 00:00:00-05:00,129.688234,133.106335,129.359378,132.916992,1311400,0.0,0.0,2024,130.106781,0.021599,3.746957
...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,2024,128.165619,0.006026,1.752578
YUM,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,2024,128.937943,-0.008678,2.000117
YUM,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,2024,127.819061,0.011310,2.237759


In [12]:
# Write your code below.
# Loading the necessary packages
import pandas as pd


# Calculate rolling average with a window of 10 days

pd_feat1 = dd_feat.groupby('ticker', group_keys=False).apply (
    lambda x: x.assign(rolling_avg_rtn=x['returns'].rolling(10).mean())
)

C:\Users\klawa\AppData\Local\Temp\ipykernel_29736\273884310.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  pd_feat1 = dd_feat.groupby('ticker', group_keys=False).apply (


In [14]:
pd_feat1.compute()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,year,Close_lag,returns,hi_lo_range,rolling_avg_rtn
ticker,,,,,,,,,,,,,
A,2013-12-02 00:00:00-05:00,34.931005,35.061659,34.695823,34.761150,2039962,0.0,0.0,2013,NaN,NaN,0.365836,NaN
A,2013-12-03 00:00:00-05:00,34.571684,34.735007,34.336502,34.578217,3462706,0.0,0.0,2013,34.761150,-0.005263,0.398505,NaN
A,2013-12-04 00:00:00-05:00,34.519425,35.166176,34.441033,35.002853,3377288,0.0,0.0,2013,34.578217,0.012280,0.725143,NaN
A,2013-12-05 00:00:00-05:00,34.852604,35.146580,34.702349,34.950596,2530939,0.0,0.0,2013,35.002853,-0.001493,0.444231,NaN
A,2013-12-06 00:00:00-05:00,35.120452,35.884789,35.120452,35.819462,4268513,0.0,0.0,2013,34.950596,0.024860,0.764337,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
YUM,2024-01-25 00:00:00-05:00,128.551783,129.027053,127.274475,128.937943,1871700,0.0,0.0,2024,128.165619,0.006026,1.752578,0.000828
YUM,2024-01-26 00:00:00-05:00,128.640891,129.403315,127.403198,127.819061,1157000,0.0,0.0,2024,128.937943,-0.008678,2.000117,0.000115
YUM,2024-01-29 00:00:00-05:00,127.828973,129.264694,127.026935,129.264694,1482100,0.0,0.0,2024,127.819061,0.011310,2.237759,0.001145


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.